In [1]:
import yfinance as yf
import pandas as pd

# List of solar companies
companies = ['FSLR', 'SPWR', 'ENPH', 'SEDG', 'CSIQ']  # Add more tickers as needed

# Fetch data for each company
stock_data = {}
for company in companies:
    stock_data[company] = yf.download(company, start='2015-01-01', end='2025-01-01')

# Combine data into a single DataFrame
all_stocks = pd.concat(
    [df.assign(company=company) for company, df in stock_data.items()],
    axis=0
).reset_index()

# Save to CSV for inspection
all_stocks.to_csv('stock_data.csv', index=False)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['SPWR']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [2]:
 stock_data = pd.read_csv('stock_data.csv')
 stock_data.head()

,Date,Close,High,Low,Open,Volume,company,Adj Close,Close.1,High.1,...,Close.3,High.3,Low.3,Open.3,Volume.3,Close.4,High.4,Low.4,Open.4,Volume.4
0,NaN,FSLR,FSLR,FSLR,FSLR,FSLR,NaN,SPWR,SPWR,SPWR,...,SEDG,SEDG,SEDG,SEDG,SEDG,CSIQ,CSIQ,CSIQ,CSIQ,CSIQ
1,2015-01-02,44.54999923706055,45.34000015258789,43.720001220703125,44.720001220703125,1873800.0,FSLR,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-01-05,41.83000183105469,43.97999954223633,41.5099983215332,43.880001068115234,3668000.0,FSLR,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-01-06,40.86000061035156,42.0099983215332,39.880001068115234,42.0099983215332,3712200.0,FSLR,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-01-07,41.75,42.470001220703125,41.400001525878906,42.0,2869800.0,FSLR,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
import yfinance as yf
import pandas as pd

# List of companies to check
companies = ['FSLR', 'SPWR', 'ENPH', 'SEDG', 'CSIQ', 'RUN']  # Add your tickers

# Dictionary to track valid and invalid companies
valid_companies = []
invalid_companies = {}

for company in companies:
    try:
        # Attempt to download data for the past year
        data = yf.download(company, start="2023-01-01", end="2024-01-01", progress=False)
        if data.empty:
            invalid_companies[company] = "No data available (possibly delisted)"
        else:
            valid_companies.append(company)
    except Exception as e:
        invalid_companies[company] = f"Error: {str(e)}"

# Save valid companies
valid_companies_df = pd.DataFrame(valid_companies, columns=['Company'])
valid_companies_df.to_csv('valid_companies.csv', index=False)
print("Valid companies saved to 'valid_companies.csv'.")

# Save invalid companies log
invalid_companies_df = pd.DataFrame(list(invalid_companies.items()), columns=['Company', 'Reason'])
invalid_companies_df.to_csv('invalid_companies.csv', index=False)
print("Invalid companies saved to 'invalid_companies.csv'.")

print(f"Valid Companies: {valid_companies}")
print(f"Invalid Companies: {invalid_companies}")


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['SPWR']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


Valid companies saved to 'valid_companies.csv'.
Invalid companies saved to 'invalid_companies.csv'.
Valid Companies: ['FSLR', 'ENPH', 'SEDG', 'CSIQ', 'RUN']
Invalid Companies: {'SPWR': 'No data available (possibly delisted)'}


In [7]:
import yfinance as yf
import pandas as pd

# List of 10 companies
companies = ['FSLR', 'ENPH', 'SEDG', 'CSIQ', 'RUN', 'JKS', 'NEE', 'TPIC', 'ORA', 'MAXN']

# Track valid and invalid companies
valid_companies = []
invalid_companies = {}

for company in companies:
    try:
        data = yf.download(company, start="2015-01-01", end="2025-01-01", progress=False)
        if data.empty:
            invalid_companies[company] = "No data available (possibly delisted)"
        else:
            valid_companies.append(company)
    except Exception as e:
        invalid_companies[company] = f"Error: {str(e)}"

# Display results
print(f"Valid Companies: {valid_companies}")
print(f"Invalid Companies: {invalid_companies}")

# Save to CSV
pd.DataFrame(valid_companies, columns=["Company"]).to_csv("valid_companies_expanded.csv", index=False)
pd.DataFrame(invalid_companies.items(), columns=["Company", "Reason"]).to_csv("invalid_companies_log.csv", index=False)


Valid Companies: ['FSLR', 'ENPH', 'SEDG', 'CSIQ', 'RUN', 'JKS', 'NEE', 'TPIC', 'ORA', 'MAXN']
Invalid Companies: {}


In [8]:
# Fetch historical data for all valid companies
all_data = {}
for company in valid_companies:
    all_data[company] = yf.download(company, start="2015-01-01", end="2025-01-01", progress=False)

# Combine all data into a single DataFrame
combined_data = pd.concat(
    [df.assign(Company=company) for company, df in all_data.items()],
    axis=0
).reset_index()

# Save combined data to CSV
combined_data.to_csv('valid_companies_stock_data.csv', index=False)
print("Historical stock data saved as 'valid_companies_stock_data.csv'.")


Historical stock data saved as 'valid_companies_stock_data.csv'.


In [9]:
# Fetch historical data for all valid companies
all_data = {}
for company in valid_companies:
    all_data[company] = yf.download(company, start="2015-01-01", end="2025-01-01", progress=False)

# Combine all data into a single DataFrame
combined_data = pd.concat(
    [df.assign(Company=company) for company, df in all_data.items()],
    axis=0
).reset_index()

# Save combined data to CSV
combined_data.to_csv('valid_companies_stock_data.csv', index=False)
print("Historical stock data saved as 'valid_companies_stock_data.csv'.")



Historical stock data saved as 'valid_companies_stock_data.csv'.


In [10]:
# Load the combined data
data = pd.read_csv('valid_companies_stock_data.csv')

# Inspect the data
print(data.info())
print(data.head())

# Check for missing values
missing_values = data.isnull().sum()
print("Missing values in each column:")
print(missing_values)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23143 entries, 0 to 23142
Data columns (total 52 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Date      23142 non-null  object
 1   Close     2517 non-null   object
 2   High      2517 non-null   object
 3   Low       2517 non-null   object
 4   Open      2517 non-null   object
 5   Volume    2517 non-null   object
 6   Company   23142 non-null  object
 7   Close.1   2517 non-null   object
 8   High.1    2517 non-null   object
 9   Low.1     2517 non-null   object
 10  Open.1    2517 non-null   object
 11  Volume.1  2517 non-null   object
 12  Close.2   2460 non-null   object
 13  High.2    2460 non-null   object
 14  Low.2     2460 non-null   object
 15  Open.2    2460 non-null   object
 16  Volume.2  2460 non-null   object
 17  Close.3   2517 non-null   object
 18  High.3    2517 non-null   object
 19  Low.3     2517 non-null   object
 20  Open.3    2517 non-null   object
 21  Volume.3  25

<ipython-input-10-9acd1f5aaabb>:2: DtypeWarning: Columns (1,2,3,4,5,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('valid_companies_stock_data.csv')


In [ ]:
import pandas as pd

# Load the dataset
file_path = 'your_dataset.csv'  # Replace with the actual file path
data = pd.read_csv(file_path)

# Step 1: Inspect the first row or column headers for company names
print("Column Headers:")
print(data.columns)

# If company names are stored in the first row, extract them
if pd.isna(data.iloc[0, 0]):  # Check if the first cell is NaN
    company_names = data.iloc[0, 1:].dropna().unique()  # Extract non-NaN values
else:
    company_names = data['company'].unique()  # Extract unique company names if they're in a column

# Display the list of companies
print("List of Companies:")
print(company_names)


In [11]:
# Drop the first row if it contains company names
data = data.iloc[1:].reset_index(drop=True)


In [12]:
# Reshape the dataset into a long format
columns_to_melt = [col for col in data.columns if 'Close' in col or 'High' in col or 'Low' in col or 'Open' in col or 'Volume' in col]

# Melt the data
melted_data = data.melt(
    id_vars=['Date', 'Company'],
    value_vars=columns_to_melt,
    var_name='Variable',
    value_name='Value'
)

# Extract the company from the Variable column (if it's embedded in column names)
melted_data['Company'] = melted_data['Variable'].str.extract(r'(\d+)$').fillna('')

# Clean up Variable names
melted_data['Variable'] = melted_data['Variable'].str.replace(r'\.\d+$', '', regex=True)


In [14]:
# Step 1: Identify the columns to melt
columns_to_melt = [col for col in data.columns if 'Close' in col or 'High' in col or 'Low' in col or 'Open' in col or 'Volume' in col]

# Step 2: Melt the data into long format
melted_data = data.melt(
    id_vars=['Date'],  # Keep Date as identifier
    value_vars=columns_to_melt,  # Columns to unpivot
    var_name='Variable',  # New column name for variable names
    value_name='Value'  # New column name for values
)

# Step 3: Extract company names from the column names (if embedded in Variable)
# Match the suffix numbers (e.g., '.1', '.2') in the Variable column
melted_data['Company_Suffix'] = melted_data['Variable'].str.extract(r'\.(\d+)$').fillna('0')

# Step 4: Replace Variable names to remove suffixes (clean up variable names)
melted_data['Variable'] = melted_data['Variable'].str.replace(r'\.\d+$', '', regex=True)

# Step 5: Map suffixes to actual company names
# Assuming you already have a mapping of suffix numbers to company names
company_mapping = {
    '0': 'FSLR',
    '1': 'ENPH',
    '2': 'SEDG',
    '3': 'CSIQ',
    '4': 'RUN',
    '5': 'JKS',
    '6': 'NEE',
    '7': 'TPIC',
    '8': 'ORA',
    '9': 'MAXN'
}

# Map suffixes to company names
melted_data['Company'] = melted_data['Company_Suffix'].map(company_mapping)

# Step 6: Drop the temporary suffix column
melted_data = melted_data.drop(columns=['Company_Suffix'])

# Step 7: Inspect the melted dataset
print(melted_data.head())
print(melted_data.info())



         Date Variable               Value Company
0  2015-01-02    Close   44.54999923706055    FSLR
1  2015-01-05    Close   41.83000183105469    FSLR
2  2015-01-06    Close   40.86000061035156    FSLR
3  2015-01-07    Close               41.75    FSLR
4  2015-01-08    Close  43.630001068115234    FSLR
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1157100 entries, 0 to 1157099
Data columns (total 4 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   Date      1157100 non-null  object
 1   Variable  1157100 non-null  object
 2   Value     115710 non-null   object
 3   Company   1157100 non-null  object
dtypes: object(4)
memory usage: 35.3+ MB
None


In [15]:
# Drop rows with missing 'Value'
cleaned_data = melted_data.dropna(subset=['Value']).copy()
print(f"Remaining rows after dropping missing values: {len(cleaned_data)}")


Remaining rows after dropping missing values: 115710


In [16]:
# Convert 'Date' to datetime
cleaned_data['Date'] = pd.to_datetime(cleaned_data['Date'], errors='coerce')

# Convert 'Value' to numeric
cleaned_data['Value'] = pd.to_numeric(cleaned_data['Value'], errors='coerce')


In [17]:
# Check the cleaned dataset
print(cleaned_data.info())
print(cleaned_data.head())


<class 'pandas.core.frame.DataFrame'>
Index: 115710 entries, 0 to 1157099
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   Date      115710 non-null  datetime64[ns]
 1   Variable  115710 non-null  object        
 2   Value     115710 non-null  float64       
 3   Company   115710 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 4.4+ MB
None
        Date Variable      Value Company
0 2015-01-02    Close  44.549999    FSLR
1 2015-01-05    Close  41.830002    FSLR
2 2015-01-06    Close  40.860001    FSLR
3 2015-01-07    Close  41.750000    FSLR
4 2015-01-08    Close  43.630001    FSLR


In [18]:
cleaned_data.to_csv('cleaned_stock_data.csv', index=False)
print("Cleaned dataset saved as 'cleaned_stock_data.csv'.")


Cleaned dataset saved as 'cleaned_stock_data.csv'.
